In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report


from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

In [2]:
df0 = pd.read_csv('../../../Week9_Start_of_Project/Technical_Assignment/From_001_Financial_BoZhang_notebook_app-BZ/0-0-all_news_pc_with_hs_by_news.csv')
df0 = df0[df0.rele_score ==1]
df0

,Unnamed: 0,datetime,headline,id,related,summary,symbol,pos,neg,neu,...,senti_prob,senti_score,rele_score,rep_score,km_label,h_label,sf1BD,non_neu_prob,price_change,date_y
0,19,1637100240,Cisco earnings should benefit from enterprise ...,72570871,CSCO,Cisco Systems Inc. could benefit from an enter...,CSCO,0.087981,0.854546,0.057473,...,-0.854546,-1,1,600,1,0,2021-11-17,0.942527,-0.004211,2021-11-17
3,22,1637079991,"Dish, Cisco Collaborate To Sell 5G To Enterprises",72558989,CSCO,DISH Network Corp (NASDAQ: DISH) collaborated ...,CSCO,0.213866,0.007034,0.779100,...,0.000000,0,1,200,0,2,2021-11-17,0.220900,-0.004211,2021-11-17
4,23,1637075955,Silicon Photonics Explained: Cisco Optics Podc...,72560643,CSCO,Episode 16 of the Cisco Optics Podcast is now ...,CSCO,0.020617,0.046141,0.933241,...,0.000000,0,1,600,1,0,2021-11-17,0.066759,-0.004211,2021-11-17
6,25,1637075835,Cisco : Bolsters Endpoint Security to Stop Thr...,72560099,CSCO,Cyberattacks hit businesses every day. Ransomw...,CSCO,0.833967,0.008128,0.157905,...,0.833967,1,1,600,1,0,2021-11-17,0.842095,-0.004211,2021-11-17
7,26,1637067979,U.S. satellite TV firm Dish ties up with Cisco...,72570924,CSCO,"U.S. satellite TV provider Dish, which is raci...",CSCO,0.792625,0.011130,0.196245,...,0.792625,1,1,200,0,2,2021-11-17,0.803755,-0.004211,2021-11-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501516,518975,1606841820,PPL Corp. stock underperforms Tuesday when com...,61564813,PPL,Shares of PPL Corp. sank 0.21% to $28.36 Tuesd...,PPL,0.054072,0.931087,0.014841,...,-0.931087,-1,1,100,3,4,2020-12-02,0.985159,0.005994,2020-12-02
501518,518977,1606755420,PPL Corp. stock underperforms Monday when comp...,61547652,PPL,Shares of PPL Corp. shed 2.37% to $28.42 Monda...,PPL,0.051392,0.934510,0.014097,...,-0.934510,-1,1,100,3,4,2020-12-01,0.985903,-0.002111,2020-12-01
501520,518979,1606497060,"PPL Corp. stock falls Friday, underperforms ma...",61525250,PPL,Shares of PPL Corp. slumped 0.95% to $29.11 Fr...,PPL,0.019904,0.965564,0.014532,...,-0.965564,-1,1,100,3,4,2020-11-30,0.985468,-0.023703,2020-11-30
501521,518982,1606236960,"PPL Corp. stock rises Tuesday, outperforms market",61486497,PPL,Shares of PPL Corp. rallied 2.30% to $30.68 Tu...,PPL,0.907800,0.061747,0.030453,...,0.907800,1,1,100,3,4,2020-11-25,0.969547,-0.042047,2020-11-25


In [3]:
df_new = df0.sort_values(by=['datetime'])
df_new

,Unnamed: 0,datetime,headline,id,related,summary,symbol,pos,neg,neu,...,senti_prob,senti_score,rele_score,rep_score,km_label,h_label,sf1BD,non_neu_prob,price_change,date_y
185628,191663,1606003200,Microsoft Fixes Xbox Series X 4K Blu-Ray Brigh...,61452415,MSFT,Hopefully some of the other niggles can be fir...,MSFT,0.083008,0.443455,0.473537,...,0.000000,0,1,600,1,0,2020-11-23,0.526463,-0.001331,2020-11-23
95941,98972,1606007452,"Why You Might Be Interested In Assurant, Inc. ...",61450009,AIZ,"Assurant, Inc. (NYSE:AIZ) is about to trade ex...",AIZ,0.055619,0.012943,0.931438,...,0.000000,0,1,100,3,4,2020-11-23,0.068562,0.010755,2020-11-23
331939,342690,1606009507,"Netflix vs. Hallmark: Who Does ""Bad"" Christmas...",61444282,NFLX,The battle of so-bad-they're-good festive films.,NFLX,0.024700,0.435375,0.539924,...,0.000000,0,1,600,1,0,2020-11-23,0.460076,-0.023800,2020-11-23
372437,384324,1606013398,"F5 Networks, Inc. (NASDAQ:FFIV) Full-Year Resu...",61450101,FFIV,Last week saw the newest full-year earnings re...,FFIV,0.051506,0.055571,0.892922,...,0.000000,0,1,600,1,0,2020-11-23,0.107078,0.006766,2020-11-23
126875,130915,1606015520,Results: Intuit Inc. Exceeded Expectations And...,61450142,INTU,"As you might know, Intuit Inc. (NASDAQ:INTU) j...",INTU,0.916904,0.051576,0.031520,...,0.916904,1,1,600,1,0,2020-11-23,0.968481,-0.012540,2020-11-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346726,357954,1637104380,Australian researchers to study how Tesla car ...,72587919,TSLA,Australia's University of Queensland (UQ) on W...,TSLA,0.127826,0.009711,0.862462,...,0.000000,0,1,400,4,1,2021-11-17,0.137538,0.032501,2021-11-17
382261,394487,1637105143,Pfizer files for U.S. authorization of promisi...,72569445,PFE,Pfizer Inc said on Tuesday it is seeking U.S. ...,PFE,0.896221,0.017938,0.085841,...,0.896221,1,1,200,0,2,2021-11-17,0.914159,0.025605,2021-11-17
292213,301477,1637105874,Twitter CFO Shows His Reluctance to Invest Com...,72572456,TWTR,Ned Segal said that Twitter’s strategy is goin...,TWTR,0.036072,0.169732,0.794196,...,0.000000,0,1,600,1,0,2021-11-17,0.205804,-0.028785,2021-11-17
217722,224801,1637106520,Nvidia Analyst Raises Price Target By 78%: 'Ch...,72579954,NVDA,"Nvidia, Inc. (NASDAQ: NVDA) shares received a ...",NVDA,0.939978,0.016690,0.043332,...,0.939978,1,1,600,1,0,2021-11-17,0.956668,-0.031189,2021-11-17


In [4]:
df_new_1 = df_new.reset_index(drop=True)
df_new_1

,Unnamed: 0,datetime,headline,id,related,summary,symbol,pos,neg,neu,...,senti_prob,senti_score,rele_score,rep_score,km_label,h_label,sf1BD,non_neu_prob,price_change,date_y
0,191663,1606003200,Microsoft Fixes Xbox Series X 4K Blu-Ray Brigh...,61452415,MSFT,Hopefully some of the other niggles can be fir...,MSFT,0.083008,0.443455,0.473537,...,0.000000,0,1,600,1,0,2020-11-23,0.526463,-0.001331,2020-11-23
1,98972,1606007452,"Why You Might Be Interested In Assurant, Inc. ...",61450009,AIZ,"Assurant, Inc. (NYSE:AIZ) is about to trade ex...",AIZ,0.055619,0.012943,0.931438,...,0.000000,0,1,100,3,4,2020-11-23,0.068562,0.010755,2020-11-23
2,342690,1606009507,"Netflix vs. Hallmark: Who Does ""Bad"" Christmas...",61444282,NFLX,The battle of so-bad-they're-good festive films.,NFLX,0.024700,0.435375,0.539924,...,0.000000,0,1,600,1,0,2020-11-23,0.460076,-0.023800,2020-11-23
3,384324,1606013398,"F5 Networks, Inc. (NASDAQ:FFIV) Full-Year Resu...",61450101,FFIV,Last week saw the newest full-year earnings re...,FFIV,0.051506,0.055571,0.892922,...,0.000000,0,1,600,1,0,2020-11-23,0.107078,0.006766,2020-11-23
4,130915,1606015520,Results: Intuit Inc. Exceeded Expectations And...,61450142,INTU,"As you might know, Intuit Inc. (NASDAQ:INTU) j...",INTU,0.916904,0.051576,0.031520,...,0.916904,1,1,600,1,0,2020-11-23,0.968481,-0.012540,2020-11-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257850,357954,1637104380,Australian researchers to study how Tesla car ...,72587919,TSLA,Australia's University of Queensland (UQ) on W...,TSLA,0.127826,0.009711,0.862462,...,0.000000,0,1,400,4,1,2021-11-17,0.137538,0.032501,2021-11-17
257851,394487,1637105143,Pfizer files for U.S. authorization of promisi...,72569445,PFE,Pfizer Inc said on Tuesday it is seeking U.S. ...,PFE,0.896221,0.017938,0.085841,...,0.896221,1,1,200,0,2,2021-11-17,0.914159,0.025605,2021-11-17
257852,301477,1637105874,Twitter CFO Shows His Reluctance to Invest Com...,72572456,TWTR,Ned Segal said that Twitter’s strategy is goin...,TWTR,0.036072,0.169732,0.794196,...,0.000000,0,1,600,1,0,2021-11-17,0.205804,-0.028785,2021-11-17
257853,224801,1637106520,Nvidia Analyst Raises Price Target By 78%: 'Ch...,72579954,NVDA,"Nvidia, Inc. (NASDAQ: NVDA) shares received a ...",NVDA,0.939978,0.016690,0.043332,...,0.939978,1,1,600,1,0,2021-11-17,0.956668,-0.031189,2021-11-17


In [5]:
#Clean Data
df_new_2 = df_new_1[['datetime','date_y','headline','symbol','price_change']]
df_new_2

,datetime,date_y,headline,symbol,price_change
0,1606003200,2020-11-23,Microsoft Fixes Xbox Series X 4K Blu-Ray Brigh...,MSFT,-0.001331
1,1606007452,2020-11-23,"Why You Might Be Interested In Assurant, Inc. ...",AIZ,0.010755
2,1606009507,2020-11-23,"Netflix vs. Hallmark: Who Does ""Bad"" Christmas...",NFLX,-0.023800
3,1606013398,2020-11-23,"F5 Networks, Inc. (NASDAQ:FFIV) Full-Year Resu...",FFIV,0.006766
4,1606015520,2020-11-23,Results: Intuit Inc. Exceeded Expectations And...,INTU,-0.012540
...,...,...,...,...,...
257850,1637104380,2021-11-17,Australian researchers to study how Tesla car ...,TSLA,0.032501
257851,1637105143,2021-11-17,Pfizer files for U.S. authorization of promisi...,PFE,0.025605
257852,1637105874,2021-11-17,Twitter CFO Shows His Reluctance to Invest Com...,TWTR,-0.028785
257853,1637106520,2021-11-17,Nvidia Analyst Raises Price Target By 78%: 'Ch...,NVDA,-0.031189


In [6]:
df_new_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257855 entries, 0 to 257854
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   datetime      257855 non-null  int64  
 1   date_y        257855 non-null  object 
 2   headline      257855 non-null  object 
 3   symbol        257855 non-null  object 
 4   price_change  257855 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 9.8+ MB


## 1. Classifying Price Change greater than 0 as 1 and less than 0 as 0 with original price changes (without 3% cutoff)

In [7]:
def to_pc_class_wo3(x):
    if x >0:
        return 1
    else:
        return 0

In [8]:
df_original_price = df_new_2
df_original_price['pc_class_wo3'] = df_original_price['price_change'].map(to_pc_class_wo3)

In [9]:
df_original_price

,datetime,date_y,headline,symbol,price_change,pc_class_wo3
0,1606003200,2020-11-23,Microsoft Fixes Xbox Series X 4K Blu-Ray Brigh...,MSFT,-0.001331,0
1,1606007452,2020-11-23,"Why You Might Be Interested In Assurant, Inc. ...",AIZ,0.010755,1
2,1606009507,2020-11-23,"Netflix vs. Hallmark: Who Does ""Bad"" Christmas...",NFLX,-0.023800,0
3,1606013398,2020-11-23,"F5 Networks, Inc. (NASDAQ:FFIV) Full-Year Resu...",FFIV,0.006766,1
4,1606015520,2020-11-23,Results: Intuit Inc. Exceeded Expectations And...,INTU,-0.012540,0
...,...,...,...,...,...,...
257850,1637104380,2021-11-17,Australian researchers to study how Tesla car ...,TSLA,0.032501,1
257851,1637105143,2021-11-17,Pfizer files for U.S. authorization of promisi...,PFE,0.025605,1
257852,1637105874,2021-11-17,Twitter CFO Shows His Reluctance to Invest Com...,TWTR,-0.028785,0
257853,1637106520,2021-11-17,Nvidia Analyst Raises Price Target By 78%: 'Ch...,NVDA,-0.031189,0


In [10]:
train_raw_wo3 = df_original_price[df_original_price['date_y']< '2021-10-01']
test_raw_wo3 = df_original_price[df_original_price['date_y']>= '2021-10-01']
len(train_raw_wo3)/len(df_original_price)

0.7738108626941498

In [11]:
EMBEDDING_FILE = '../from_xiaojian/001_model/crawl-300d-2M.vec'

In [12]:
X_train_wo3 = train_raw_wo3['headline'].values
y_train_wo3 = train_raw_wo3['pc_class_wo3'].values
X_test_wo3 = test_raw_wo3['headline'].values
y_test_wo3 = test_raw_wo3['pc_class_wo3'].values

In [13]:
y_train_wo3_2d = keras.utils.to_categorical(y_train_wo3,num_classes=2)
y_test_wo3_2d = keras.utils.to_categorical(y_test_wo3,num_classes=2)

In [14]:
max_features_wo3 = 3000
maxlen_wo3 = 100
embed_size_wo3 = 300

In [16]:
tokenizer_wo3 = text.Tokenizer(num_words=max_features_wo3)
tokenizer_wo3.fit_on_texts(list(X_train_wo3) + list(X_test_wo3))
X_train_wo3 = tokenizer_wo3.texts_to_sequences(X_train_wo3)
X_test_wo3 = tokenizer_wo3.texts_to_sequences(X_test_wo3)
x_train_wo3 = sequence.pad_sequences(X_train_wo3, maxlen=maxlen_wo3)
x_test_wo3 = sequence.pad_sequences(X_test_wo3, maxlen=maxlen_wo3)


In [17]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [18]:
word_index_wo3 = tokenizer_wo3.word_index
nb_words_wo3 = min(max_features_wo3, len(word_index_wo3))
embedding_matrix_wo3 = np.zeros((nb_words_wo3, embed_size_wo3))
for word, i in word_index_wo3.items():
    if i >= max_features_wo3: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_wo3[i] = embedding_vector


In [19]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


In [24]:
def get_model_wo3():
    inp = Input(shape=(maxlen_wo3, ))
    x = Embedding(max_features_wo3, embed_size_wo3, weights=[embedding_matrix_wo3])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(2, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [25]:
model_wo3 = get_model_wo3()

2022-03-10 23:19:07.688858: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
batch_size_wo3 = 32
epochs_wo3 = 4

In [30]:
X_tra_wo3, X_val_wo3, y_tra_wo3, y_val_wo3 = train_test_split(x_train_wo3, y_train_wo3_2d, train_size=0.95, random_state=233)
RocAuc_wo3 = RocAucEvaluation(validation_data=(X_val_wo3, y_val_wo3), interval=1)


In [31]:
hist_wo3 = model_wo3.fit(X_tra_wo3, y_tra_wo3, batch_size=batch_size_wo3, epochs=epochs_wo3, validation_data=(X_val_wo3, y_val_wo3),
                 callbacks=[RocAuc_wo3], verbose=2)

Epoch 1/4

 ROC-AUC - epoch: 1 - score: 0.592934 

5924/5924 - 482s - loss: 0.6848 - accuracy: 0.5472 - val_loss: 0.6749 - val_accuracy: 0.5688 - 482s/epoch - 81ms/step
Epoch 2/4

 ROC-AUC - epoch: 2 - score: 0.611579 

5924/5924 - 507s - loss: 0.6677 - accuracy: 0.5750 - val_loss: 0.6709 - val_accuracy: 0.5634 - 507s/epoch - 86ms/step
Epoch 3/4

 ROC-AUC - epoch: 3 - score: 0.623648 

5924/5924 - 493s - loss: 0.6453 - accuracy: 0.6026 - val_loss: 0.6657 - val_accuracy: 0.5785 - 493s/epoch - 83ms/step
Epoch 4/4

 ROC-AUC - epoch: 4 - score: 0.638168 

5924/5924 - 486s - loss: 0.6229 - accuracy: 0.6256 - val_loss: 0.6590 - val_accuracy: 0.5945 - 486s/epoch - 82ms/step


### 1.1 Predictions for Training with Original Price Changes

In [33]:
y_train_pred_wo3 = model_wo3.predict(x_train_wo3, batch_size = 1024)
y_train_predclasses_wo3 = y_train_pred_wo3.argmax(axis=1)

train_accu_wo3 = accuracy_score(y_train_wo3, y_train_predclasses_wo3)
train_class_report_wo3 = classification_report(y_train_wo3, y_train_predclasses_wo3)
train_rocauc_wo3 = roc_auc_score(y_train_wo3, y_train_predclasses_wo3)

print(f'The training accuracy score is {train_accu_wo3}')
print(f'The training classification report is {train_class_report_wo3}')
print(f'The training roc_auc_score is {train_rocauc_wo3}')


The training accuracy score is 0.6565095148122347
The training classification report is               precision    recall  f1-score   support

           0       0.68      0.52      0.59     95361
           1       0.64      0.78      0.70    104170

    accuracy                           0.66    199531
   macro avg       0.66      0.65      0.65    199531
weighted avg       0.66      0.66      0.65    199531

The training roc_auc_score is 0.6509399491080884


### 1.2 Predictions for Testing with Original Price Changes

In [34]:
y_test_pred_wo3 = model_wo3.predict(x_test_wo3, batch_size=1024)
y_test_predclasses_wo3 = y_test_pred_wo3.argmax(axis=1)

test_accu_wo3 = accuracy_score(y_test_wo3, y_test_predclasses_wo3)
test_class_report_wo3 = classification_report(y_test_wo3, y_test_predclasses_wo3)
test_rocauc_wo3 = roc_auc_score(y_test_wo3, y_test_predclasses_wo3)

print(f'The testing accuracy score is {test_accu_wo3}')
print(f'The testing classification report is {test_class_report_wo3}')
print(f'The testing roc_auc_score is {test_rocauc_wo3}')


The training accuracy score is 0.5221521157670942
The training classification report is               precision    recall  f1-score   support

           0       0.47      0.42      0.44     26284
           1       0.56      0.61      0.58     32040

    accuracy                           0.52     58324
   macro avg       0.51      0.51      0.51     58324
weighted avg       0.52      0.52      0.52     58324

The training roc_auc_score is 0.5128727981554027


## 2. Classifying Price Change greater than 0.03 as 1 and less than 0.03 as 0 after truncating data of +-3% changes

In [35]:
df_bigChange = df_new_2[abs(df_new_2['price_change'])>=0.03]

In [36]:
def to_pc_class_w3(x):
    if x >=0.03:
        return 1
    else:
        return 0

In [37]:
df_bigChange['pc_class_w3'] = df_bigChange['price_change'].map(to_pc_class_w3)

In [38]:
df_bigChange

,datetime,date_y,headline,symbol,price_change,pc_class_wo3,pc_class_w3
7,1606019581,2020-11-23,SHAREHOLDER ALERT: Pomerantz Law Firm Investig...,C,0.031946,1,1
10,1606028412,2020-11-23,Marriott appoints new EMEA president [Construc...,MAR,0.031942,1,1
39,1606032868,2020-11-23,Delta casts doubts on New York-London travel c...,DAL,0.044708,1,1
46,1606034933,2020-11-23,GM ups bet on EVs as investors swoon for elect...,GM,0.040195,1,1
51,1606041774,2020-11-23,Delta hints at the possibility of purchasing B...,DAL,0.044708,1,1
...,...,...,...,...,...,...,...
257832,1637100265,2021-11-17,"PayPal taps Treasury, O'Melveny alum Patel as ...",PYPL,-0.043585,0,0
257839,1637101800,2021-11-17,APA Corporation Announces Successful Flow Test...,APA,-0.066172,0,0
257840,1637101806,2021-11-17,Tesla bounces back as electric car stocks rally,TSLA,0.032501,1,1
257850,1637104380,2021-11-17,Australian researchers to study how Tesla car ...,TSLA,0.032501,1,1


In [39]:
train_raw_w3 = df_bigChange[df_bigChange['date_y']< '2021-10-01']
test_raw_w3 = df_bigChange[df_bigChange['date_y']>= '2021-10-01']
len(train_raw_w3)/len(df_bigChange)

0.7471189555496

In [40]:
X_train_w3 = train_raw_w3['headline'].values
y_train_w3 = train_raw_w3['pc_class_w3'].values
X_test_w3 = test_raw_w3['headline'].values
y_test_w3 = test_raw_w3['pc_class_w3'].values

In [41]:
y_train_w3_2d = keras.utils.to_categorical(y_train_w3,num_classes=2)
y_test_w3_2d = keras.utils.to_categorical(y_test_w3,num_classes=2)

In [42]:
max_features_w3 = 3000
maxlen_w3 = 100
embed_size_w3 = 300

In [45]:
tokenizer_w3 = text.Tokenizer(num_words=max_features_w3)
tokenizer_w3.fit_on_texts(list(X_train_w3) + list(X_test_w3))
X_train_w3 = tokenizer_w3.texts_to_sequences(X_train_w3)
X_test_w3 = tokenizer_w3.texts_to_sequences(X_test_w3)
x_train_w3 = sequence.pad_sequences(X_train_w3, maxlen=maxlen_w3)
x_test_w3 = sequence.pad_sequences(X_test_w3, maxlen=maxlen_w3)

In [46]:
word_index_w3 = tokenizer_w3.word_index
nb_words_w3 = min(max_features_w3, len(word_index_w3))
embedding_matrix_w3 = np.zeros((nb_words_w3, embed_size_w3))
for word, i in word_index_w3.items():
    if i >= max_features_w3: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_w3[i] = embedding_vector

In [47]:
def get_model_w3():
    inp = Input(shape=(maxlen_w3, ))
    x = Embedding(max_features_w3, embed_size_w3, weights=[embedding_matrix_w3])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(2, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [48]:
model_w3 = get_model_w3()

In [49]:
batch_size_w3 = 32
epochs_w3 = 6

In [50]:
X_tra_w3, X_val_w3, y_tra_w3, y_val_w3 = train_test_split(x_train_w3, y_train_w3_2d, train_size=0.95, random_state=233)
RocAuc_w3 = RocAucEvaluation(validation_data=(X_val_w3, y_val_w3), interval=1)


In [51]:
hist_w3 = model_w3.fit(X_tra_w3, y_tra_w3, batch_size=batch_size_w3, epochs=epochs_w3, validation_data=(X_val_w3, y_val_w3),
                 callbacks=[RocAuc_w3], verbose=2)

Epoch 1/6

 ROC-AUC - epoch: 1 - score: 0.681887 

580/580 - 46s - loss: 0.6538 - accuracy: 0.6021 - val_loss: 0.6396 - val_accuracy: 0.6066 - 46s/epoch - 79ms/step
Epoch 2/6

 ROC-AUC - epoch: 2 - score: 0.724595 

580/580 - 44s - loss: 0.5700 - accuracy: 0.6848 - val_loss: 0.5902 - val_accuracy: 0.6465 - 44s/epoch - 75ms/step
Epoch 3/6

 ROC-AUC - epoch: 3 - score: 0.746192 

580/580 - 42s - loss: 0.5191 - accuracy: 0.7210 - val_loss: 0.5801 - val_accuracy: 0.6609 - 42s/epoch - 73ms/step
Epoch 4/6

 ROC-AUC - epoch: 4 - score: 0.760501 

580/580 - 42s - loss: 0.4757 - accuracy: 0.7473 - val_loss: 0.5862 - val_accuracy: 0.6598 - 42s/epoch - 73ms/step
Epoch 5/6

 ROC-AUC - epoch: 5 - score: 0.764473 

580/580 - 45s - loss: 0.4367 - accuracy: 0.7682 - val_loss: 0.5974 - val_accuracy: 0.6957 - 45s/epoch - 78ms/step
Epoch 6/6

 ROC-AUC - epoch: 6 - score: 0.771541 

580/580 - 44s - loss: 0.3908 - accuracy: 0.7954 - val_loss: 0.6013 - val_accuracy: 0.6844 - 44s/epoch - 76ms/step


### 2.1 Predictions for Training with Truncating 3% Price Changes

In [53]:
y_train_pred_w3 = model_w3.predict(x_train_w3, batch_size = 1024)
y_train_predclasses_w3 = y_train_pred_w3.argmax(axis=1)

train_accu_w3 = accuracy_score(y_train_w3, y_train_predclasses_w3)
train_class_report_w3 = classification_report(y_train_w3, y_train_predclasses_w3)
train_rocauc_w3 = roc_auc_score(y_train_w3, y_train_predclasses_w3)

print(f'The training accuracy score is {train_accu_w3}')
print(f'The training classification report is {train_class_report_w3}')
print(f'The training roc_auc_score is {train_rocauc_w3}')


The training accuracy score is 0.8370400737931741
The training classification report is               precision    recall  f1-score   support

           0       0.81      0.84      0.82      8740
           1       0.86      0.84      0.85     10774

    accuracy                           0.84     19514
   macro avg       0.84      0.84      0.84     19514
weighted avg       0.84      0.84      0.84     19514

The training roc_auc_score is 0.836880983926471


### 2.2 Predictions for Testing with Truncating 3% Price Changes

In [55]:
y_test_pred_w3 = model_w3.predict(x_test_w3, batch_size=1024)
y_test_predclasses_w3 = y_test_pred_w3.argmax(axis=1)

test_accu_w3 = accuracy_score(y_test_w3, y_test_predclasses_w3)
test_class_report_w3 = classification_report(y_test_w3, y_test_predclasses_w3)
test_rocauc_w3 = roc_auc_score(y_test_w3, y_test_predclasses_w3)

print(f'The testing accuracy score is {test_accu_w3}')
print(f'The testing classification report is {test_class_report_w3}')
print(f'The testing roc_auc_score is {test_rocauc_w3}')


The training accuracy score is 0.501438304314913
The training classification report is               precision    recall  f1-score   support

           0       0.43      0.52      0.47      2835
           1       0.57      0.49      0.53      3770

    accuracy                           0.50      6605
   macro avg       0.50      0.50      0.50      6605
weighted avg       0.51      0.50      0.50      6605

The training roc_auc_score is 0.5036875640323916
